In [2]:
# %load_ext autoreload
import importlib
# import autoreload
import pandas as pd
import os
import sys
import pwd
import numpy as np
from pathlib import Path
from tqdm import tqdm, trange
from tqdm.contrib.concurrent import process_map
import matplotlib.pyplot as plt
import GEOparse
sys.path.append('../')

import scanpy as sc
from scipy import sparse as sp
import scipy as spy
from scipy.stats import linregress, entropy
from meth import tools as mtools
from meth import analytic_tools as anatools
# %autoreload 2
# plt.style.use('pics/meth.mplstyle')
plt.style.use('../Stochastic aging/pics/meth.mplstyle')

In [4]:
def add_chr_pos(df, col_from='index'):
    df['chr'] = df['index'].apply(lambda x: x.split('_')[0])
    df['pos_start'] = df['index'].apply(lambda x: int(x.split('_')[1]))
    df['pos_end'] = df['index'].apply(lambda x: int(x.split('_')[1]) + 10)
    return df

# Petkovich aging dynamics 7 types

In [35]:
df_raw = pd.read_csv('../Stochastic aging/dump/Petkovich_stochastic_models_full_list.csv')
df_raw.head()

,index,mask_nonzero,r_age,p_age,entropic_BA_b,entropic_BA_x0,entropic_down_BA_b,entropic_down_BA_x0,entropic_up_BA_b,entropic_up_BA_x0,...,ent_insignificant_up_BA_x0,ent_insignificant_down_BA_b,ent_insignificant_down_BA_x0,ent_insignificant_BA_b,ent_insignificant_BA_x0,petkovich_clock,pd,pm,q,x0
0,chr10_100015516,True,0.247693,0.004339,0.0,0.0,0.0,0.0,0.0,0.0,...,-2.095952,0.0,0.0,4.450867,-2.095952,0.0,0.003840,0.00001,0.118046,0.136509
1,chr10_100015517,False,0.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,NaN,NaN,NaN,0.065552
2,chr10_100015540,True,0.372971,0.000155,0.0,0.0,0.0,0.0,0.0,0.0,...,-2.923920,0.0,0.0,2.968078,-2.923920,0.0,0.002958,0.00001,0.055041,0.081446
3,chr10_100015541,False,0.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,NaN,NaN,NaN,0.075994
4,chr10_100015553,True,0.202238,0.019109,0.0,0.0,0.0,0.0,0.0,0.0,...,-2.724293,0.0,0.0,6.117220,-2.724293,0.0,0.001219,0.00001,0.070708,0.082098


(1976056, 36)

In [36]:


df = add_chr_pos(df_raw)


In [37]:
df_raw.head()

,index,mask_nonzero,r_age,p_age,entropic_BA_b,entropic_BA_x0,entropic_down_BA_b,entropic_down_BA_x0,entropic_up_BA_b,entropic_up_BA_x0,...,ent_insignificant_BA_b,ent_insignificant_BA_x0,petkovich_clock,pd,pm,q,x0,chr,pos_start,pos_end
0,chr10_100015516,True,0.247693,0.004339,0.0,0.0,0.0,0.0,0.0,0.0,...,4.450867,-2.095952,0.0,0.003840,0.00001,0.118046,0.136509,chr10,100015516,100015526
1,chr10_100015517,False,0.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,NaN,NaN,NaN,0.065552,chr10,100015517,100015527
2,chr10_100015540,True,0.372971,0.000155,0.0,0.0,0.0,0.0,0.0,0.0,...,2.968078,-2.923920,0.0,0.002958,0.00001,0.055041,0.081446,chr10,100015540,100015550
3,chr10_100015541,False,0.000000,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,NaN,NaN,NaN,0.075994,chr10,100015541,100015551
4,chr10_100015553,True,0.202238,0.019109,0.0,0.0,0.0,0.0,0.0,0.0,...,6.117220,-2.724293,0.0,0.001219,0.00001,0.070708,0.082098,chr10,100015553,100015563


In [40]:
bioages_for_plotting = ['1', '2', '3','4','5','6','7']#'entropic_down_BA', 'entropic_up_BA', 'antientropic_down_BA', 'antientropic_up_BA',
#                        'constant_BA',  'antient_insignificant_up_BA', 'antient_insignificant_down_BA', 
 #                       'ent_insignificant_up_BA', 'ent_insignificant_down_BA', 'petkovich_clock']

# df = dsets['Mice_C57BL6'].obs.copy()
df['col4'] = "*"
df['col5'] = 0.
df['strand'] = "*"

for i, bioage in enumerate(bioages_for_plotting):
    if bioage == '1':
        idx = df[f"p_age"].values > 0.05
        idx = np.logical_and(idx, df['x0'].values < 0.2)
    elif bioage == '2':
        idx = df[f"entropic_up_BA_b"].abs().values > 1e-3
    elif bioage == '3':
        idx = df[f"antientropic_up_BA_b"].abs().values > 1e-3
    elif bioage == '4':
        idx = df[f"p_age"].values > 0.05
        idx = np.logical_and(idx, df['x0'].values > 0.8)
    elif bioage == '5':
        idx = df[f"entropic_down_BA_b"].abs().values > 1e-3
    elif bioage == '6':
        idx = df[f"antientropic_down_BA_b"].abs().values > 1e-3
    elif bioage == '7':
        idx = df[f"p_age"].values > 0.05
        idx = np.logical_and(idx, np.logical_and(df['x0'].values > 0.4, df['x0'].values < 0.6))
    df.loc[df.index[idx], 'col4'] = bioage
df = df[df['col4'] != '*']

df[['chr', 'pos_start', 'pos_end', 'col4', 'col5', 'strand', 'r_age', 'p_age']].to_csv(f'../Stochastic aging/data/GSE80672/GSE80672_regions_7types.csv', index=None, header=None, sep='\t')

/var/folders/v_/gtsrj3xs0_131ltxwg51r_f40000gn/T/ipykernel_31935/3921614170.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['col4'] = "*"
/var/folders/v_/gtsrj3xs0_131ltxwg51r_f40000gn/T/ipykernel_31935/3921614170.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['col5'] = 0.
/var/folders/v_/gtsrj3xs0_131ltxwg51r_f40000gn/T/ipykernel_31935/3921614170.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

In [23]:
bioages_for_plotting = ['entropic_down_BA', 'entropic_up_BA', 'antientropic_down_BA', 'antientropic_up_BA',
                        'constant_BA',  'antient_insignificant_up_BA', 'antient_insignificant_down_BA', 
                        'ent_insignificant_up_BA', 'ent_insignificant_down_BA', 'petkovich_clock']

# df = dsets['Mice_C57BL6'].obs.copy()
df['col4'] = "*"
df['col5'] = 0.
df['strand'] = "*"

for i, bioage in enumerate(bioages_for_plotting):
    if bioage == 'petkovich_clock':
        idx = df[f"{bioage}"].abs().values > 1e-3
    else:
        idx = df[f"{bioage}_b"].abs().values > 1e-3
    df.loc[df.index[idx], 'col4'] = bioage
df = df[df['col4'] != '*']

# df[['chr', 'pos_start', 'pos_end', 'col4', 'col5', 'strand', 'r_age', 'p_age', 'entropic_BA_b']].to_csv(f'data/GSE80672/GSE80672_regions.csv', index=None, header=None, sep='\t')
    
# df[['chr', 'pos_start', 'pos_end', 'col4', 'col5', 'strand', 'r_age', 'p_age', 'entropic_BA_b', 'entropic_BA_x0',  'antientropic_BA_b', 'antientropic_BA_x0']].to_csv(f'data/GSE80672/GSE80672_{regions[i]}_regions.csv', index=None, header=None)

In [24]:
# df[['chr', 'pos_start', 'pos_end', 'col4', 'col5', 'strand', 'r_age', 'p_age', 'entropic_BA_b']].to_csv(f'data/GSE80672/GSE80672_regions.csv', index=None, header=None, sep='\t')
df[['chr', 'pos_start', 'pos_end', 'col4', 'col5', 'strand', 'r_age', 'p_age']].to_csv(f'../Stochastic aging/data/GSE80672/GSE80672_regions_full.csv', index=None, header=None, sep='\t')

In [25]:
df_old = pd.read_csv('../Stochastic aging/data/GSE80672/GSE80672_regions.csv', sep='\t')
df_old.head()

,chr10,100015510,100015520,ent_insignificant_up_BA,0.0,*,0.20020815972683317,0.020373399929582096,0.0.1
0,chr10,100015540,100015550,ent_insignificant_up_BA,0.0,*,0.441400,3.216111e-07,0.0
1,chr10,100015550,100015560,constant_BA,0.0,*,0.085392,3.175559e-01,0.0
2,chr10,100015600,100015610,constant_BA,0.0,*,0.045083,6.008984e-01,0.0
3,chr10,100015650,100015660,ent_insignificant_up_BA,0.0,*,0.364834,3.602859e-05,0.0
4,chr10,100015680,100015690,ent_insignificant_up_BA,0.0,*,0.418594,5.907741e-06,0.0


In [26]:
df = pd.read_csv('../Stochastic aging/data/GSE80672/GSE80672_regions_full.csv', sep='\t')
df.head()

,chr10,100015510,100015520,ent_insignificant_up_BA,0.0,*,0.2002081597268331,0.020373399929582
0,chr10,100015540,100015550,ent_insignificant_up_BA,0.0,*,0.441400,3.216111e-07
1,chr10,100015550,100015560,constant_BA,0.0,*,0.085392,3.175559e-01
2,chr10,100015600,100015610,constant_BA,0.0,*,0.045083,6.008984e-01
3,chr10,100015650,100015660,ent_insignificant_up_BA,0.0,*,0.364834,3.602859e-05
4,chr10,100015680,100015690,ent_insignificant_up_BA,0.0,*,0.418594,5.907741e-06


In [ ]:
# entropic_regions.reset_index(inplace=True)
regions = ['entropic', 'antientropic']
for i, df in enumerate([entropic_regions, antientropic_regions]):
    df.reset_index(inplace=True)
    df['chr'] = df['index'].apply(lambda x: x.split('_')[0])
    df['pos_start'] = df['index'].apply(lambda x: int(x.split('_')[1]))
    df['pos_end'] = df['index'].apply(lambda x: int(x.split('_')[1]) + 10)
    df['col4'] = df['r_age'] > 1e-3
    df['col5'] = 0.
    df['strand'] = "*"
    df[['chr', 'pos_start', 'pos_end', 'col4', 'col5', 'strand', 'r_age', 'p_age', 'entropic_BA_b']].to_csv(f'data/GSE80672/GSE80672_{regions[i]}_regions.csv', index=None, header=None, sep='\t')
    # df[['chr', 'pos_start', 'pos_end', 'col4', 'col5', 'strand', 'r_age', 'p_age', 'entropic_BA_b', 'entropic_BA_x0',  'antientropic_BA_b', 'antientropic_BA_x0']].to_csv(f'data/GSE80672/GSE80672_{regions[i]}_regions.csv', index=None, header=None)

# Coreg clusters

In [58]:
df_raw = pd.read_csv('dump_for_enrichment_large_strict_coreg_working_on_502.csv')
df = add_chr_pos(df_raw)
df.head()

,index,hypermethylated_with_age,hypomethylated_with_age,co_regulated_cluster,stochastic_cluster,chr,pos_start,pos_end
0,chr10_100017344,NaN,NaN,NaN,NaN,chr10,100017344,100017354
1,chr10_100023398,True,False,False,True,chr10,100023398,100023408
2,chr10_100025890,True,False,False,True,chr10,100025890,100025900
3,chr10_100025896,True,False,False,True,chr10,100025896,100025906
4,chr10_100053911,NaN,NaN,NaN,NaN,chr10,100053911,100053921


In [59]:
df = df[df['chr'] != 'chrMT']

In [60]:
df['col4'] = "*"
df['col5'] = 0
df['strand'] = "*"
df['col6'] = 0
df['col7'] = 0

/var/folders/v_/gtsrj3xs0_131ltxwg51r_f40000gn/T/ipykernel_7008/4023787084.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['col4'] = "*"
/var/folders/v_/gtsrj3xs0_131ltxwg51r_f40000gn/T/ipykernel_7008/4023787084.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['col5'] = 0
/var/folders/v_/gtsrj3xs0_131ltxwg51r_f40000gn/T/ipykernel_7008/4023787084.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [61]:
df['col4'] = 'Background'
df.loc[df['hypermethylated_with_age'] == True,'col4'] = 'Hypermethylated'
df.loc[df['hypomethylated_with_age'] == True,'col4'] = 'Hypomethylated'

/var/folders/v_/gtsrj3xs0_131ltxwg51r_f40000gn/T/ipykernel_7008/3689626997.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['col4'] = 'Background'
/Users/tarkhov/opt/anaconda3/envs/py310/lib/python3.10/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [62]:
df[['chr', 'pos_start', 'pos_end', 'col4', 'col5', 'strand','col6','col7']].to_csv(f'../Stochastic aging/data/GSE80672/scDNAm_aging_hypohyper_meth.csv', index=None, header=None, sep='\t')

In [63]:
df['col4'] = 'Background'
df.loc[df['co_regulated_cluster'] == True,'col4'] = 'Co-regulated'
df.loc[df['stochastic_cluster'] == True,'col4'] = 'Stochastic'

/var/folders/v_/gtsrj3xs0_131ltxwg51r_f40000gn/T/ipykernel_7008/2844980324.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['col4'] = 'Background'


In [64]:
df[['chr', 'pos_start', 'pos_end', 'col4', 'col5', 'strand','col6','col7']].to_csv(f'../Stochastic aging/data/GSE80672/scDNAm_aging_coreg_stoch_meth.csv', index=None, header=None, sep='\t')

In [65]:
df[['chr', 'pos_start', 'pos_end', 'col4', 'col5', 'strand','col6','col7']]

,chr,pos_start,pos_end,col4,col5,strand,col6,col7
0,chr10,100017344,100017354,Background,0,*,0,0
1,chr10,100023398,100023408,Stochastic,0,*,0,0
2,chr10,100025890,100025900,Stochastic,0,*,0,0
3,chr10,100025896,100025906,Stochastic,0,*,0,0
4,chr10,100053911,100053921,Background,0,*,0,0
...,...,...,...,...,...,...,...,...
103785,chrY,90829072,90829082,Background,0,*,0,0
103786,chrY,946514,946524,Co-regulated,0,*,0,0
103787,chrY,9958213,9958223,Background,0,*,0,0
103788,chrY,999103,999113,Background,0,*,0,0


In [66]:
df = pd.read_csv(f'../Stochastic aging/data/GSE80672/GSE80672_regions_7types.csv', sep='\t')
df.head()

,chr10,100015517,100015527,1,0.0,*,0.0.1,1.0
0,chr10,100015541,100015551,1,0.0,*,0.000000,1.000000
1,chr10,100015554,100015564,1,0.0,*,0.033357,0.709672
2,chr10,100015601,100015611,1,0.0,*,0.016966,0.852851
3,chr10,100015602,100015612,1,0.0,*,0.075109,0.390211
4,chr10,100015640,100015650,1,0.0,*,0.000000,1.000000


In [71]:
df = pd.read_csv(f'../Stochastic aging/data/GSE80672/scDNAm_aging_coreg_stoch_meth.csv', sep='\t', header=None)
df.head()

,0,1,2,3,4,5,6,7
0,chr10,100017344,100017354,Background,0,*,0,0
1,chr10,100023398,100023408,Stochastic,0,*,0,0
2,chr10,100025890,100025900,Stochastic,0,*,0,0
3,chr10,100025896,100025906,Stochastic,0,*,0,0
4,chr10,100053911,100053921,Background,0,*,0,0


In [68]:
df = pd.read_csv(f'../Stochastic aging/data/GSE80672/scDNAm_aging_hypohyper_meth.csv', sep='\t', header=None)
df.head()

,0,1,2,3,4,5,6,7
0,chr10,100017344,100017354,Background,0,*,0,0
1,chr10,100023398,100023408,Hypermethylated,0,*,0,0
2,chr10,100025890,100025900,Hypermethylated,0,*,0,0
3,chr10,100025896,100025906,Hypermethylated,0,*,0,0
4,chr10,100053911,100053921,Background,0,*,0,0


In [73]:
df[3].value_counts().index

Index(['Background', 'Stochastic', 'Co-regulated'], dtype='object')